## EasyOCR

- 介紹
- 安裝與官網
- 使用方式
- 實例

## 介紹
### What is EasyOCR?

EasyOCR 是一個第三方函式庫用於圖像文字識別

其特色有

- 支持70種語言

- 使用簡易

- 允許使用外部訓練集

## 安裝與官網

採用 `pip` 安裝

最新版:

``` bash
pip install easyocr
```

穩定版:

``` bash
pip install git+git://github.com/jaidedai/easyocr.git
```

官網:
https://github.com/JaidedAI/EasyOCR/blob/master/README.md


## 使用方式

以官網範例做例子

``` python
import easyocr
reader = easyocr.Reader(['ch_sim','en'], gpu=False) # 這僅需跑一次讓模塊載入記憶體
result = reader.readtext('chinese.jpg')
```

上方 `easyocr.Reader()` 內資料為
1. `['ch_sim','en']` : 語言資料，這邊例子是中文簡體與英文
2. `gpu=False` : 這邊代表運算是否用GPU

<img src="/JaidedAI/EasyOCR/blob/master/examples/example2.png" alt="example2.png">

在最上方事例正常會讀取出
``` bash
[([[189, 75], [469, 75], [469, 165], [189, 165]], '愚园路', 0.3754989504814148),
 ([[86, 80], [134, 80], [134, 128], [86, 128]], '西', 0.40452659130096436),
 ([[517, 81], [565, 81], [565, 123], [517, 123]], '东', 0.9989598989486694),
 ([[78, 126], [136, 126], [136, 156], [78, 156]], '315', 0.8125889301300049),
 ([[514, 126], [574, 126], [574, 156], [514, 156]], '309', 0.4971577227115631),
 ([[226, 170], [414, 170], [414, 220], [226, 220]], 'Yuyuan Rd.', 0.8261902332305908),
 ([[79, 173], [125, 173], [125, 213], [79, 213]], 'W', 0.9848111271858215),
 ([[529, 173], [569, 173], [569, 213], [529, 213]], 'E', 0.8405593633651733)]
```

上方共識別了8組資料，以第一筆為例，其主要參數為
1. `[[189, 75], [469, 75], [469, 165], [189, 165]]` : 識別座標4個定位點
2. `'愚园路'` : 判定的字

## 實際操作

在這個例子中，我引入CV2用來識別紅色的字其設定為

1. 需有個`box.png`在這個資料夾目錄

2. 需安裝繁體中文訓練集 (首次執行會自動下載到根目錄`./model`)

3. 這邊我是識別紅色的字，如果`color_detect`為`True`即可做色彩處理

4. 圖片預處理後會存在`box_conv.png`

5. 圖片探測框圖像化存在`box_detect.png`

實際展示可以去github位置看 : https://github.com/whats2000/coding-stuff-I-make-from-learning/tree/main/pythons/lab6

In [2]:
import numpy as np
from PIL import Image, ImageDraw
import easyocr
import cv2

reader = easyocr.Reader(['ch_tra'], gpu=True, model_storage_directory='../model')


def detect_png(color_detect=False):
    pre_img = cv2.cvtColor(cv2.imread(r'box.png'), cv2.COLOR_BGR2HSV)

    if color_detect:
        low_hsv = np.array([0, 43, 46])
        high_hsv = np.array([10, 255, 255])

        pre2_img = cv2.inRange(pre_img, lowerb=low_hsv, upperb=high_hsv)
    else:
        pre2_img = pre_img

    cv2.imwrite('box_conv.png', pre2_img)

    detect_text = reader.readtext(r'box_conv.png')

    img = Image.open(r'box_conv.png')
    draw = ImageDraw.Draw(img)

    for t in detect_text:
        draw.rectangle((tuple(t[0][0]), tuple(t[0][2])), fill=None, outline='red', width=2)

    img.save("box_detect.png")

    print('偵測字串: ', detect_text)


detect_png(False)


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


偵測字串:  [([[0, 39], [245, 39], [245, 77], [0, 77]], '倒計時::1分 54秒', 0.7812354200434776)]
